# ETL del dataset user_reviews

Importacion de librerias 

In [10]:
import pandas as pd
import gzip
from pandas import json_normalize

Carga de dataset

In [27]:
with gzip.open('../Jsons/users_items.json.gz', 'rt', encoding='utf-8') as f:
    df = [eval(line) for line in f]

users_items = pd.DataFrame(df)

Visualizacion de los primeros 5 registros

In [7]:
users_items.head() 

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


Eliminacion de columas iniecesarias

In [88]:
df_items = users_items.drop(columns=['items_count','user_url','steam_id']) 

Cuento los nulos de cada columa para ver como tratar los datos

In [51]:
df_items.isnull().sum() 

user_id     0
steam_id    0
items       0
dtype: int64

Desanidado de columnas items

In [47]:
# Normalización de los datos
df_items_normalized = json_normalize(df_items['items'].explode().tolist())
# Concatenación de los datos normalizados con los datos originales
df_items = pd.concat([df_items.drop(['items'], axis=1), df_items_normalized], axis=1)

Se elimina la columna '0' ya que se creo al desanidar la columna items y la columna 'playtime_2weeks' ya que no se necesita

In [100]:
df_items.drop(columns=['0','playtime_2weeks'], inplace=True) 

Tratamiento de duplicados 

In [102]:
# Conteo de duplicados
df_items.duplicated().sum() 

59209

In [107]:
df_items['item_id'] = df_items['item_id'].astype('Int64') # cambio de tipo de dato
df_items['user_id'] = df_items['user_id'].astype('str') # cambio de tipo de dato

Elimino los duplicados que comparten con la columna 'user_id' y 'item_id'

In [108]:
df_items = df_items.drop_duplicates(subset=['user_id','item_id'])

0

Hago una visualización de 10 muestras para seguir tratando los datos 

In [149]:
df_items.head(10) 

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0
5,76561197970982479,60,Ricochet,0.0
6,76561197970982479,70,Half-Life,0.0
7,76561197970982479,130,Half-Life: Blue Shift,0.0
8,76561197970982479,300,Day of Defeat: Source,4733.0
9,76561197970982479,240,Counter-Strike: Source,1853.0


Elimino las filas en caso de que la columna 'playtime_forever' esté vacia o tenga valor 0 ya que me interesa guardar jugadores sin horas jugadas

In [150]:
df_items = df_items.dropna(subset=['playtime_forever'])
df_items = df_items[df_items['playtime_forever'] != 0]

Ultima visualizacion para seguir tratando los datos o guardar

In [155]:
df_items

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
2,76561197970982479,30,Day of Defeat,7.0
8,76561197970982479,300,Day of Defeat: Source,4733.0
9,76561197970982479,240,Counter-Strike: Source,1853.0
10,76561197970982479,3830,Psychonauts,333.0
...,...,...,...,...
5170007,76561198329548331,304930,Unturned,677.0
5170008,76561198329548331,227940,Heroes & Generals,43.0
5170011,76561198329548331,388490,One Way To Die: Steam Edition,3.0
5170012,76561198329548331,521570,You Have 10 Seconds 2,4.0


Guardo el dataset steam_games como csv para utilizarlo en la API

In [156]:
df_items.to_csv('../Data/df_items.csv', index=False)